In [1]:
# our usual pylab import
%pylab --no-import-all inline

Populating the interactive namespace from numpy and matplotlib


# Goal

For background, see [Mapping Census Data](http://www.udel.edu/johnmack/frec682/census/), including the 
[scan of the 10-question form](http://www.udel.edu/johnmack/frec682/census/census_form.png).  Keep in mind what people were asked and the range of data available in the census.

Using the census API to get an understanding of some of the geographic entities in the **2010 census**.  We'll specifically be using the variable `P0010001`, the total population.  

What you will do in this notebook:

 * Sum the population of the **states** (or state-like entity like DC) to get the total population of the **nation**
 * Add up the **counties** for each **state** and validate the sums
 * Add up the **census tracts** for each **county** and validate the sums
 
We will make use of `pandas` in this notebook.

I often have the following [diagram](http://www.census.gov/geo/reference/pdfs/geodiagram.pdf) in mind to help understand the relationship among entities.  Also use the [list of example URLs](http://api.census.gov/data/2010/sf1/geo.html)  -- it'll come in handy.

<a href="http://www.flickr.com/photos/raymondyee/12297467734/" title="Census Geographic Hierarchies by Raymond Yee, on Flickr"><img src="http://farm4.staticflickr.com/3702/12297467734_af8882d310_c.jpg" width="618" height="800" alt="Census Geographic Hierarchies"></a>

# Working out the geographical hierarchy for Cafe Milano

It's helpful to have a concrete instance of a place to work with, especially when dealing with rather intangible entities like census tracts, block groups, and blocks.  You can use the [American FactFinder](http://factfinder2.census.gov/faces/nav/jsf/pages/index.xhtml) site to look up for any given US address the corresponding census geographies.  

Let's use Cafe Milano in Berkeley as an example. You can verify the following results by typing in the address into http://factfinder2.census.gov/faces/nav/jsf/pages/searchresults.xhtml?refresh=t.  

https://www.evernote.com/shard/s1/sh/dc0bfb96-4965-4fbf-bc28-c9d4d0080782/2bd8c92a045d62521723347d62fa2b9d

2522 Bancroft Way, BERKELEY, CA, 94704

* State: California
* County: Alameda County
* County Subdivision: Berkeley CCD, Alameda County, California
* Census Tract: Census Tract 4228, Alameda County, California
* Block Group: Block Group 1, Census Tract 4228, Alameda County, California
* Block: Block 1001, Block Group 1, Census Tract 4228, Alameda County, California



In [2]:
# YouTube video I made on how to use the American Factfinder site to look up addresses
from IPython.display import YouTubeVideo
YouTubeVideo('HeXcliUx96Y')

In [3]:
#  standard numpy, pandas, matplotlib imports

import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame, Series, Index
import pandas as pd

In [4]:
# check that CENSUS_KEY is defined
import census
import us

import requests

import settings
assert settings.CENSUS_KEY is not None

The census documentation has example URLs but needs your API key to work.  In this notebook, we'll use the IPython notebook HTML display mechanism to help out.


In [5]:
c = census.Census(key=settings.CENSUS_KEY)

Note:  we can use `c.sf1` to access 2010 census (SF1: Census Summary File 1 (2010, 2000, 1990) available in API -- 2010 is the default)

see documentation: [sunlightlabs/census](https://github.com/sunlightlabs/census)

# Summing up populations by state    

Let's make a `DataFrame` named `states_df` with columns `NAME`, `P0010001` (for population), and `state` (to hold the FIPS code).  **Make sure to exclude Puerto Rico.**

In [6]:
# call the API and instantiate `df`
df = DataFrame(c.sf1.get('NAME,P0010001', geo={'for':'state:*'}))
# convert the population to integer
df['P0010001'] = df['P0010001'].astype(np.int)
df.head()

,NAME,P0010001,state
0,Alabama,4779736,01
1,Alaska,710231,02
2,Arizona,6392017,04
3,Arkansas,2915918,05
4,California,37253956,06


You can filter Puerto Rico (PR) in a number of ways -- use the way you're most comfortable with. 

Optional fun: filter PR in the following way

* calculate a `np.array` holding the the fips of the states
* then use [numpy.in1d](http://docs.scipy.org/doc/numpy/reference/generated/numpy.in1d.html), which is a analogous to the [in](http://stackoverflow.com/a/3437130/7782) operator to test membership in a list

In [7]:
states_fips = np.array([state.fips for state in us.states.STATES])
states_df = df[np.in1d(df.state,states_fips)]

If `states_df` is calculated properly, the following asserts will pass silently.

In [8]:
# check that we have three columns
assert set(states_df.columns) == set((u'NAME', u'P0010001', u'state'))

# check that the total 2010 census population is correct
assert np.sum(states_df.P0010001) == 308745538 

# check that the number of states+DC is 51
assert len(states_df) == 51

# Counties

Looking at http://api.census.gov/data/2010/sf1/geo.html, we see

    state-county: http://api.census.gov/data/2010/sf1?get=P0010001&for=county:*
    
if we want to grab all counties in one go, or you can grab counties state-by-state:

    http://api.census.gov/data/2010/sf1?get=P0010001&for=county:*&in=state:06
    
for all counties in the state with FIPS code `06` (which is what state?)

In [9]:
# Here's a way to use translate 
# http://api.census.gov/data/2010/sf1?get=P0010001&for=county:*
# into a call using the census.Census object

r = c.sf1.get('NAME,P0010001', geo={'for':'county:*'})

# ask yourself what len(r) means and what it should be
len(r)

3221

In [10]:
# let's try out one of the `census` object convenience methods
# instead of using `c.sf1.get`

r = c.sf1.state_county('NAME,P0010001',census.ALL,census.ALL)
r

[{u'NAME': u'Autauga County',
  u'P0010001': u'54571',
  u'county': u'001',
  u'state': u'01'},
 {u'NAME': u'Baldwin County',
  u'P0010001': u'182265',
  u'county': u'003',
  u'state': u'01'},
 {u'NAME': u'Barbour County',
  u'P0010001': u'27457',
  u'county': u'005',
  u'state': u'01'},
 {u'NAME': u'Bibb County',
  u'P0010001': u'22915',
  u'county': u'007',
  u'state': u'01'},
 {u'NAME': u'Blount County',
  u'P0010001': u'57322',
  u'county': u'009',
  u'state': u'01'},
 {u'NAME': u'Bullock County',
  u'P0010001': u'10914',
  u'county': u'011',
  u'state': u'01'},
 {u'NAME': u'Butler County',
  u'P0010001': u'20947',
  u'county': u'013',
  u'state': u'01'},
 {u'NAME': u'Calhoun County',
  u'P0010001': u'118572',
  u'county': u'015',
  u'state': u'01'},
 {u'NAME': u'Chambers County',
  u'P0010001': u'34215',
  u'county': u'017',
  u'state': u'01'},
 {u'NAME': u'Cherokee County',
  u'P0010001': u'25989',
  u'county': u'019',
  u'state': u'01'},
 {u'NAME': u'Chilton County',
  u'P001000

In [11]:
# convert the json from the API into a DataFrame
# coerce to integer the P0010001 column

df = DataFrame(r)
df['P0010001'] = df['P0010001'].astype('int')

# display the first records
df.head()

,NAME,P0010001,county,state
0,Autauga County,54571,001,01
1,Baldwin County,182265,003,01
2,Barbour County,27457,005,01
3,Bibb County,22915,007,01
4,Blount County,57322,009,01


In [12]:
# calculate the total population 
# what happens when you google the number you get?

np.sum(df['P0010001'])

312471327

In [13]:
# often you can use dot notation to access a DataFrame column
df.P0010001.head()

0     54571
1    182265
2     27457
3     22915
4     57322
Name: P0010001, dtype: int64

In [14]:
# let's filter out PR -- what's the total population now
sum(df[np.in1d(df.state, states_fips)].P0010001)

308745538

In [15]:
# fall back to non-Pandas solution if you need ton
np.sum([int(county['P0010001']) for county in r if county['state'] in states_fips])

308745538

In [16]:
# construct counties_df with only 50 states + DC
counties_df = df[np.in1d(df.state, states_fips)]
len(counties_df)

3143

In [17]:
set(counties_df.columns) == set(df.columns)

True

Check properties of `counties_df`

In [18]:
# number of counties
assert len(counties_df) == 3143 #3143 county/county-equivs in US

In [19]:
# check that the total population by adding all counties == population by adding all states

assert np.sum(counties_df['P0010001']) == np.sum(states_df.P0010001)

In [20]:
# check we have same columns between counties_df and df
set(counties_df.columns) == set(df.columns)

True

# Using FIPS code as the Index

From [Mapping Census Data](http://www.udel.edu/johnmack/frec682/census/):

* Each state (SUMLEV = 040) has a 2-digit FIPS ID; Delaware's is 10.
* Each county (SUMLEV = 050) within a state has a 3-digit FIPS ID, appended to the 2-digit state ID. New Castle County, Delaware, has FIPS ID 10003.
* Each Census Tract (SUMLEV = 140) within a county has a 6-digit ID, appended to the county code. The Tract in New Castle County DE that contains most of the the UD campus has FIPS ID 10003014502.
* Each Block Group (SUMLEV = 150) within a Tract has a single digit ID appended to the Tract ID. The center of campus in the northwest corner of the tract is Block Group100030145022.
* Each Block (SUMLEV = 750) within a Block Group is identified by three more digits appended to the Block Group ID. Pearson Hall is located in Block 100030145022009.

In [21]:
# take a look at the current structure of counties_df

counties_df.head()

,NAME,P0010001,county,state
0,Autauga County,54571,001,01
1,Baldwin County,182265,003,01
2,Barbour County,27457,005,01
3,Bibb County,22915,007,01
4,Blount County,57322,009,01


In [22]:
# reindex states_df by state FIPS
# http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.set_index.html

states_df.set_index(keys='state', inplace=True)
states_df.head()

,NAME,P0010001
state,,
01,Alabama,4779736
02,Alaska,710231
04,Arizona,6392017
05,Arkansas,2915918
06,California,37253956


In [23]:
# display the result of using set_index
counties_df.head()

,NAME,P0010001,county,state
0,Autauga County,54571,001,01
1,Baldwin County,182265,003,01
2,Barbour County,27457,005,01
3,Bibb County,22915,007,01
4,Blount County,57322,009,01


In [24]:
# http://manishamde.github.io/blog/2013/03/07/pandas-and-python-top-10/#create

counties_df['FIPS'] = counties_df.apply(lambda s:s['state'] + s['county'], axis=1)
counties_df.set_index('FIPS', inplace=True)

/Users/raymondyee/anaconda/envs/myenv/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [25]:
counties_df.head()

,NAME,P0010001,county,state
FIPS,,,,
01001,Autauga County,54571,001,01
01003,Baldwin County,182265,003,01
01005,Barbour County,27457,005,01
01007,Bibb County,22915,007,01
01009,Blount County,57322,009,01


In [26]:
counties_df.groupby('state').sum().head()

,P0010001
state,
01,4779736
02,710231
04,6392017
05,2915918
06,37253956


In [27]:
states_df.P0010001.head()

state
01     4779736
02      710231
04     6392017
05     2915918
06    37253956
Name: P0010001, dtype: int64

In [28]:
# now we're ready to compare for each state, if you add all the counties, do you get the same
# population?
# not that you can do .agg('sum') instead of .sum()
# look at http://pandas.pydata.org/pandas-docs/dev/groupby.html to learn more about agg

np.all(states_df.P0010001 == counties_df.groupby('state').agg('sum').P0010001)

True

# Counties in California

Let's look at home: California state and Alameda County

In [29]:
# boolean indexing to pull up California
states_df[states_df.NAME == 'California']

,NAME,P0010001
state,,
06,California,37253956


In [30]:
# use .ix -- most general indexing 
# http://pandas.pydata.org/pandas-docs/dev/indexing.html#different-choices-for-indexing-loc-iloc-and-ix
states_df.ix['06']

NAME        California
P0010001      37253956
Name: 06, dtype: object

In [31]:
# California counties

counties_df[counties_df.state=='06']

,NAME,P0010001,county,state
FIPS,,,,
06001,Alameda County,1510271,001,06
06003,Alpine County,1175,003,06
06005,Amador County,38091,005,06
06007,Butte County,220000,007,06
06009,Calaveras County,45578,009,06
06011,Colusa County,21419,011,06
06013,Contra Costa County,1049025,013,06
06015,Del Norte County,28610,015,06
06017,El Dorado County,181058,017,06


In [32]:
counties_df[counties_df.NAME == 'Alameda County']

,NAME,P0010001,county,state
FIPS,,,,
06001,Alameda County,1510271,001,06


In [33]:
counties_df[counties_df.NAME == 'Alameda County']['P0010001']

FIPS
06001    1510271
Name: P0010001, dtype: int64

Different ways to read off the population of Alameda County -- still looking for the best way

In [34]:
counties_df[counties_df.NAME == 'Alameda County']['P0010001'].to_dict().values()[0]

1510271

In [35]:
list(counties_df[counties_df.NAME == 'Alameda County']['P0010001'].iteritems())[0][1]

1510271

In [36]:
int(counties_df[counties_df.NAME == 'Alameda County']['P0010001'].values)

1510271

If you know the FIPS code for Alameda County, just read off the population using `.ix`

In [37]:
# this is like accessing a cell in a spreadsheet -- row, col

ALAMEDA_COUNTY_FIPS = '06001'

counties_df.ix[ALAMEDA_COUNTY_FIPS,'P0010001']

1510271

# Reading off all the tracts in Alameda County

In [38]:
counties_df.ix[ALAMEDA_COUNTY_FIPS,'county']

u'001'

In [39]:
# http://api.census.gov/data/2010/sf1/geo.html
# state-county-tract

geo = {'for': 'tract:*', 
       'in': 'state:%s county:%s' % (us.states.CA.fips, 
                                     counties_df.ix[ALAMEDA_COUNTY_FIPS,'county'])}
        
r = c.sf1.get('NAME,P0010001', geo=geo)

In [40]:
#use state_county_tract to make a DataFrame

alameda_county_tracts_df = DataFrame(r)
alameda_county_tracts_df['P0010001'] = alameda_county_tracts_df['P0010001'].astype('int')
alameda_county_tracts_df['FIPS'] = alameda_county_tracts_df.apply(lambda s: s['state']+s['county']+s['tract'], axis=1)
alameda_county_tracts_df.head()

,NAME,P0010001,county,state,tract,FIPS
0,Census Tract 4001,2937,001,06,400100,06001400100
1,Census Tract 4002,1974,001,06,400200,06001400200
2,Census Tract 4003,4865,001,06,400300,06001400300
3,Census Tract 4004,3703,001,06,400400,06001400400
4,Census Tract 4005,3517,001,06,400500,06001400500


In [41]:
alameda_county_tracts_df.apply(lambda s: s['state']+s['county']+s['tract'], axis=1)

0      06001400100
1      06001400200
2      06001400300
3      06001400400
4      06001400500
5      06001400600
6      06001400700
7      06001400800
8      06001400900
9      06001401000
10     06001401100
11     06001401200
12     06001401300
13     06001401400
14     06001401500
15     06001401600
16     06001401700
17     06001401800
18     06001402200
19     06001402400
20     06001402500
21     06001402600
22     06001402700
23     06001402800
24     06001402900
25     06001403000
26     06001403100
27     06001403300
28     06001403400
29     06001403501
          ...     
331    06001450742
332    06001450743
333    06001450744
334    06001450745
335    06001450746
336    06001450750
337    06001450751
338    06001450752
339    06001451101
340    06001451102
341    06001451201
342    06001451202
343    06001451300
344    06001451401
345    06001451403
346    06001451404
347    06001451501
348    06001451503
349    06001451504
350    06001451505
351    06001451506
352    06001

In [42]:
alameda_county_tracts_df.P0010001.sum()

1510271

In [43]:
# Cafe Milano is in tract 4228
MILANO_TRACT_ID = '422800'
alameda_county_tracts_df[alameda_county_tracts_df.tract==MILANO_TRACT_ID]

,NAME,P0010001,county,state,tract,FIPS
133,Census Tract 4228,8368,001,06,422800,06001422800


# Using Generators to yield all the tracts in the country

http://www.jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

In [44]:
import time
import us

from itertools import islice

def census_tracts(variable=('NAME','P0010001'), sleep_time=1.0):
    
    for state in us.states.STATES:
        print state
        for tract in c.sf1.get(variable, 
                    geo={'for':"tract:*", 
                        'in':'state:{state_fips}'.format(state_fips=state.fips)
                        }):
            yield tract
        # don't hit the API more than once a second    
        time.sleep(sleep_time)
 
# limit the number of tracts we crawl for until we're reading to get all of them        
tracts_df = DataFrame(list(islice(census_tracts(), 100)))
tracts_df['P0010001'] = tracts_df['P0010001'].astype('int')


Alabama


In [45]:
tracts_df.head()

,NAME,P0010001,county,state,tract
0,Census Tract 201,1912,001,01,020100
1,Census Tract 202,2170,001,01,020200
2,Census Tract 203,3373,001,01,020300
3,Census Tract 204,4386,001,01,020400
4,Census Tract 205,10766,001,01,020500


# Compare with Tabulations

We can compare the total number of tracts we calculate to:

https://www.census.gov/geo/maps-data/data/tallies/tractblock.html

and

https://www.census.gov/geo/maps-data/data/docs/geo_tallies/Tract_Block2010.txt